In [29]:
import json
from urllib.request import Request, urlopen

### Browse Schools
#### This API request returns a single school or a list of K-12 schools (and related directory & GreatSchools 1-10 Rating data) based on the parameters passed to it.

In [36]:
def api_puller(zipcode):
    """
    The api_puller function accepts the zipcode input (as a string) and returns a dictionary of the schools located in that zipcode

    :param zipcode:
    :return: export_json
    """
    apikey = 'lB6mbjJnVv1S307NPl4B61pPfDXH9Gxn5QxpdTQu'
    req = Request('https://gs-api.greatschools.org/schools?zip=' + zipcode)
    req.add_header('X-API-Key', apikey)
    content = urlopen(req).read()
    export_json = json.loads(content)
    return export_json

### Nearby Schools
#### This API request returns a list of K-12 schools (and related directory & GreatSchools 1-10 Rating data) closest to an identified location.
##### Not built out because we are using the "Browse Schools" API Call

In [ ]:
nearbyReq = Request('https://gs-api.greatschools.org/nearby-schools?lat=37.7940627&lon=-122.2680029&limit=3&distance=5')
nearbyReq.add_header('X-API-Key', apiKey)
nearbyContent = urlopen(nearbyReq).read()

In [35]:
zipInput = '94607'
dataTest = api_puller(zipInput)

## Check API Key Status

In [40]:
def APIStatus():
    apikey = 'lB6mbjJnVv1S307NPl4B61pPfDXH9Gxn5QxpdTQu'
    APIreq = Request('https://gs-api.greatschools.org/usage?api_key=' + apikey)
    APIreq.add_header('X-API-Key', apikey)
    APIcontent = urlopen(APIreq).read()
    exportAPI = json.loads(APIcontent)
    return exportAPI

In [39]:
APIStatus()

{'calls_made': 11,
 'remaining_quota': 989,
 'time_period': '2023-02-01 to 2023-02-28'}